In [4]:
import pandas as pd
import numpy as np
import scipy.stats as st
import os

pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 400)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from pprint import pprint
from pathlib import Path

In [5]:
star_wars = pd.read_csv('./Data/StarWars.csv', sep=';')
star_wars.head()

,respondentID,seen,fan,gender,age_group,income,loc,yoda,princess_leia_organa
0,3292879998,да,да,М,18-29,NaN,Южно-Атлантический,очень нравится,очень нравится
1,3292879538,нет,NaN,М,18-29,0 - 24.999,Юго-Западный центральный,NaN,NaN
2,3292765271,да,нет,М,18-29,0 - 24.999,Северо-Западный центральный,не знаю такого персонажа,нравится
3,3292763116,да,да,М,18-29,100.000 - 149.999,Северо-Западный центральный,очень нравится,очень нравится
4,3292731220,да,да,М,18-29,100.000 - 149.999,Северо-Западный центральный,нравится,нравится


# Описание данных

- respondentID — идентификационный номер респондента
- seen — ответ на вопрос, видел ли респондент хотя бы один фильм из серии «Звездных войн» (да/нет)
- fan — ответ на вопрос, считает ли респондент себя фанатом серии фильмов (да/нет)
- gender — пол респондента (М/Ж)
- age_group — возрастная группа, к которой принадлежит респондент (18-29, 30-44, 45-60, > 60)
- income — доход респондента в долларах США (0 - 24.999, 25.000 - 49.999, 50.000 - 99.999, 100.000 - 149.999, 150.000+)
- loc — регион проживания респондента (в США)
- yoda — отношение респондента к персонажу серии (Йоде)
- princess_leia_organa — отношение респондента к персонажу серии (принцессе Лее Органе)

# Задания

### Обратим внимание на переменные `gender` и `fan`. Какую гипотезу мы можем протестировать, используя хи-квадрат? Выберите один верный ответ.

- [ ] Распределение наблюдений по категориям выбранных переменных ошибочно
- [x] Распределение наблюдений по категориям выбранных переменных независимо
- [ ] Распределение наблюдений по категориям выбранных переменных корректно
- [ ] Распределение наблюдений по категориям выбранных переменных отсутствует

### Постройте таблицу сопряженности для переменных `gender` и `fan`. 
Укажите, верно ли следующее утверждение: 120 мужчин являются фанатами серии фильмов «Звездные войны».

In [16]:
ct_o = star_wars.pivot_table(values='respondentID', index='gender', columns='fan', aggfunc='count')
ct_o

fan,да,нет
gender,,
Ж,238,159
М,303,120


### Предположим, что мы тестируем гипотезу о том, что распределение наблюдений по категориям выбранных нами переменных независимо. 
Чему будет равно количество степеней свободы, рассчитанное по построенной в предыдущем задании таблице сопряженности? Выберите один верный ответ.

In [17]:
(nrows, ncols) = ct_o.shape
dof = (nrows - 1) * (ncols - 1)
dof

1

### Что означает равенство уровня значимости пяти процентам? 
Выберите один верный ответ.

- [x] Вероятность ошибки первого рода равна 5%
- [ ] Вероятность ошибки первого рода равна 95%
- [ ] Вероятность ошибки второго рода равна 5%
- [ ] Вероятность ошибки второго рода равна 95%

### Чему будет равно критическое значение критерия хи-квадрат для указанных выше параметров (уровня значимости 5% и верного числа степеней свободы), при котором мы сможем отвергнуть нулевую гипотезу? 
Выберите один верный ответ.

In [18]:
3.84

3.84

### Чему равна оценка вероятности того, что человек является фанатом серии фильмов? 
Введите ответ в виде десятичной дроби, округлите до двух знаков после запятой.

In [29]:
chi, p = st.chisquare(f_obs=star_wars['fan'].value_counts().to_numpy(), ddof=1)
print(round(chi, 2))

85.91


### Чему равна теоретическая частота для женщин, которые являются фанатами серии фильмов? 
В ответ запишите число, округлите до целого.

$
f_e = \frac{f_cf_r}{n}
$

In [54]:
def f_e(ct_o, idx, col):
    return round(ct_o.loc[:, col].sum() * ct_o.loc[idx, :].sum() / ct_o.to_numpy().sum())

In [55]:
f_e(ct_o, 'Ж', 'да')

262

### Чему равна теоретическая частота для мужчин, которые не являются фанатами серии фильмов?

В ответ запишите число, округлите до целого.

In [56]:
f_e(ct_o, 'М', 'нет')

144

### Чему равен критерий хи-квадрат? 
Введите ответ в виде десятичной дроби, округлите до двух знаков после запятой.

In [58]:
chi_square = scipy.stats.chi2_contingency(observed=ct_o,correction=False)[0]

print(round(chi_square, 2))

12.45
